# Data Cleaning: Handling missing Values

In [ ]:
# We’re going to be looking at how to deal with missing values :D
# To get started, Download the required Dataset which I’m going to use in this tutorial

In [ ]:
# Import the libs we will use
import pandas as pd
import numpy as np

In [ ]:
# read in all our data

# Detailed NFL Play-by-Play Data 2009-2018
nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv", low_memory=False)

nfl_data.head(5)

In [ ]:
# getting a sample
sample= nfl_data.sample(4)

sample

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
198718,2013-10-20,2013102012,17,2,1.0,00:06,1,1806.0,4.0,DEN,...,NaN,0.858491,0.141509,0.858491,0.141509,0.141509,0.000000,NaN,NaN,2013
4027,2009-09-20,2009092003,5,2,4.0,11:51,12,2511.0,6.0,ARI,...,NaN,0.319042,0.680958,0.476127,0.523873,0.319042,0.157085,NaN,NaN,2009
282662,2015-10-11,2015101103,18,4,2.0,03:35,4,215.0,46.0,ATL,...,NaN,0.191746,0.808254,0.198890,0.801110,0.808254,-0.007144,NaN,NaN,2015
171802,2012-12-16,2012121600,18,4,1.0,12:16,13,736.0,43.0,CHI,...,NaN,0.071993,0.928007,0.080356,0.919644,0.928007,-0.008362,NaN,NaN,2012


In [ ]:
# get the number of missing data point per column
missing_values_count = nfl_data.isnull().sum()

# look at the number of missing points in the first ten columns
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

In [ ]:
# That seems like a lot! It might be helpful to see what percentage of the values in our dataset 
# were missing to give us a better sense of the scale of this problem

# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

24.87214126835169

In [ ]:
# Almost a quarter of the cells in this dataset are empty! 

# Figure out why the data is missing 

Is this value missing because it was not recorded or because it does not exist? 

If a value is missing because it does not exist 
    Example: 
        the height of the oldest child of someone who doesn’t have any children
        PenalizedTeam: falta el campo porque si no hubo penalización
It doesn’t make sense to try and guess what it might be


If a value is missing because it was not recorded
    Example: 
        temperature a certain hour 
        TimeSecs: cantidad de segundos que quedan en el juego cuando se realizó la jugada.
You can try to guess what it might have been based on the other values in that column and row


If you’re doing very careful data analysis, this is the point at which you’d look at each column individually to figure out the best strategy for filling those missing values.

# Drop missing values

In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                    [2,      3,      5],
                   [np.nan, 4,      6]])

df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,2.0,3.0,5
3,NaN,4.0,6


In [ ]:
# This will tell us the total number of non null observations present including the total number of entries. 
# Once number of entries isn’t equal to number of non null observations, we can begin to suspect missing values.

df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       3 non-null      float64
 1   1       3 non-null      float64
 2   2       4 non-null      int64  
dtypes: float64(2), int64(1)
memory usage: 224.0 bytes


In [ ]:
# This will tell us the total number of NaN in or data.
df.isnull().sum()

0    1
1    1
2    0
dtype: int64

In [ ]:
# remove all the rows that contain a missing value
df.dropna()

,0,1,2
1,2.0,3.0,5
2,2.0,3.0,5


In [ ]:
# remove columns the rows that contain a missing value 
# axis{0 or ‘index’, 1 or ‘columns’}
df.dropna(axis=1)

,2
0,2
1,5
2,5
3,6


In [ ]:
df.dropna(axis='columns')

,2
0,2
1,5
2,5
3,6


In [ ]:
# remove all the rows with at least
# thresh 	Require that many non-NA values.
df.dropna(axis='rows', thresh=3)

,0,1,2
1,2.0,3.0,5
2,2.0,3.0,5


# Filling missing values

In [ ]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,2.0,3.0,5
3,NaN,4.0,6


In [ ]:
# One option we have is to specify what we want the NaN values to be replaced with. 
# Here, I'm saying that I would like to replace all the NaN values with 0.
df.fillna(0)

,0,1,2
0,1.0,0.0,2
1,2.0,3.0,5
2,2.0,3.0,5
3,0.0,4.0,6


In [ ]:
# calculate the mean
# skip the Na values while finding the mean 
df.mean(axis = 0, skipna = True) 

0    1.666667
1    3.333333
2    4.500000
dtype: float64

In [ ]:
# use the mean to fill the gaps in that column
df[1]=df[1].fillna(3.3)
df

,0,1,2
0,1.0,3.3,2
1,2.0,3.0,5
2,2.0,3.0,5
3,NaN,4.0,6


In [ ]:
# We can specify a backfill use next valid observation to fill the actual gap.
df.fillna(method='bfill')

,0,1,2
0,1.0,3.3,2
1,2.0,3.0,5
2,2.0,3.0,5
3,NaN,4.0,6


In [ ]:
# We can specify a forward-fill use previous valid observation to fill the actual gap
df.fillna(method='ffill')

,0,1,2
0,1.0,3.3,2
1,2.0,3.0,5
2,2.0,3.0,5
3,2.0,4.0,6


## Fill missing values with KNN

In [ ]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/horse-colic.csv'
data = pd.read_csv(url, header=None, na_values='?')

In [ ]:
data.head()

In [ ]:
# summarize the number of rows with missing values for each column
for i in range(data.shape[1]):
	# count number of rows with missing values
	n_miss = data[[i]].isnull().sum()
	perc = n_miss / data.shape[0] * 100
	print('> %d, Missing: %d (%.1f%%)' % (i, n_miss, perc))

In [ ]:
# split into input and output elements
data = data.values
ix = [i for i in range(data.shape[1]) if i != 23]
X, y = data[:, ix], data[:, 23]

In [ ]:
X

In [ ]:
y

In [ ]:
# Impute missing values with KNN
from sklearn.impute import KNNImputer

# define imputer
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

# fit on the dataset
imputer.fit(X)

# transform the dataset
Xtrans = imputer.transform(X)

In [ ]:
# print total missing
print('Missing: %d' % sum(np.isnan(Xtrans).flatten()))

# Drop duplicates

dataframe.drop_duplicates()
    Paramenters
        subset: Subset takes a column or list of column label. It’s default value is none. 
        keep: keep is to control how to consider duplicate value. It has only three distinct value and default is ‘first’.

            If ‘first’, it considers first value as unique and rest of the same values as duplicate.
            If ‘last’, it considers last value as unique and rest of the same values as duplicate.
            If False, it consider all of the same values as duplicates
        inplace: Boolean values, removes rows with duplicates if True.

    Return type: DataFrame with removed duplicate rows depending on Arguments passed.

In [ ]:
# dropping ALL duplicte values in the column 0, only keep the first one
df.drop_duplicates(subset =[0], 
                     keep = 'first') 

,0,1,2
0,1.0,3.3,2
1,2.0,3.0,5
3,NaN,4.0,6
